# Day 16: Ticket Translation 
As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a rule like class: 1-3 or 5-7 means that one of the fields in every ticket is named class and can be any value in the ranges 1-3 or 5-7 (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:
```
.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
```
Here, ? represents text in a language you don't understand. This ticket might be represented as 101,102,103,104,301,302,303,401,402,403; of course, the actual train tickets you're looking at are much more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are completely invalid; these are tickets that contain values which aren't valid for any field. Ignore your ticket for now.

For example, suppose you have the following notes:
```
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
```
It doesn't matter which position corresponds to which field; you can identify invalid nearby tickets by considering only whether tickets contain values that are not valid for any field. In this example, the values on the first nearby ticket are all valid for at least one field. This is not true of the other three nearby tickets: the values 4, 55, and 12 are are not valid for any field. Adding together all of the invalid values produces your ticket scanning error rate: 4 + 55 + 12 = 71.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?

In [1]:
with open("input.txt", "r") as f:
    input_data = f.read().split("\n")

In [2]:
label_rownum = {x:i for i, x in enumerate(input_data) if x in ["your ticket:", "nearby tickets:"]}

In [3]:
rules_raw = input_data[:label_rownum["your ticket:"]-1]

In [4]:
your_ticket = [int(x) for x in input_data[label_rownum["your ticket:"]+1].split(",")]

In [5]:
neighbor_tickets = [[int(xx) for xx in x.split(",")]  for x in input_data[label_rownum['nearby tickets:']+1:]]

In [6]:
rule_dict = {}
for r in rules_raw:
    key, valid_value_row = r.split(": ")
    valid_range = valid_value_row.split(" or ")
    valid_range_bound = [[int(xx) for xx in x.split("-")] for x in valid_range]
    rule_dict[key] = valid_range_bound

In [7]:
def valid_ticket_value_rule(ticket_value, rule):
    return (rule[0][0] <= ticket_value <= rule[0][1]) or (rule[1][0] <= ticket_value <= rule[1][1])

In [8]:
def valid_ticket_value_any_rules(ticket_value, rule_dict):
    return any(valid_ticket_value_rule(ticket_value, rule) for rule in rule_dict.values())

In [9]:
def valid_ticket(ticket, rule_dict):
    for ticket_value in ticket:
        if not valid_ticket_value_any_rules(ticket_value, rule_dict):
            return False, ticket_value
    return True, None

In [10]:
def all_invalid_tickets(tickets, rule_dict):
    result = []
    for i, t in enumerate(tickets):
        single = valid_ticket(t, rule_dict)
        if single[0] is False:
            result.append([i, single[1]])
    return result

In [11]:
def scanning_error_rate(neighbor_tickets, rule_dict):
    return sum([x[1] for x in all_invalid_tickets(neighbor_tickets, rule_dict)])

In [12]:
scanning_error_rate(neighbor_tickets, rule_dict)

32835

# Part Two 

Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

For example, suppose you have the following notes:
```
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
```
Based on the nearby tickets in the above example, the first position must be row, the second position must be class, and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?



In [13]:
def all_valid_tickets(neighbor_tickets, rule_dict,  your_ticket):
    invalid_label = [x[0] for x in all_invalid_tickets(neighbor_tickets, rule_dict)]
    
    result = []
    for i, t in enumerate(neighbor_tickets):
        if i in invalid_label:
            continue
        result = result + [t]
    result = result + [your_ticket]
    return result

In [14]:
def valid_ticket_value_rule(ticket_value, rule):
    return (rule[0][0] <= ticket_value <= rule[0][1]) or (rule[1][0] <= ticket_value <= rule[1][1])

In [15]:
def valid_ticket_column_rule(ticket_column, rule):
    return all(valid_ticket_value_rule(tv, rule) for tv in ticket_column)

In [16]:
def valid_all_columns_rules(all_tickets, rule_dict):
    result = {}
    for col in range(len(all_tickets[0])):
        ticket_column = [x[col] for x in all_tickets]
        result[col] = []
        for rule in rule_dict.keys():
            if valid_ticket_column_rule(ticket_column, rule_dict[rule]):
                result[col] = result[col] + [rule]
    return result

In [17]:
def match_col_key(neighbor_tickets, rule_dict,  your_ticket):
    all_tickets = all_valid_tickets(neighbor_tickets, rule_dict,  your_ticket)
    col_key_valid = valid_all_columns_rules(all_tickets, rule_dict)
    key_col_match = []
    while len(key_col_match) < len(rule_dict):
        col = {len(v):k for k, v in col_key_valid.items()}[1]
        key = col_key_valid[col][0]
        key_col_match.append((col,key))
        
        del col_key_valid[col]
        col_key_valid = {k: [x for x in v if x != key] for k, v in col_key_valid.items()}
    
    return key_col_match

In [18]:
def departure_prod(neighbor_tickets, rule_dict,  your_ticket):
    match = match_col_key(neighbor_tickets, rule_dict,  your_ticket)
    departure_index = [x[0] for x in match if x[1].startswith("departure")]
    result = 1
    for i in departure_index:
        result = result * your_ticket[i]
    return result

In [19]:
departure_prod(neighbor_tickets, rule_dict,  your_ticket)

514662805187